In [265]:
import math
import numpy as np

In [266]:
def project_with_scale(lat, lon, scale):
    siny = np.sin(lat * np.pi / 180)
    siny = min(max(siny, -0.9999), 0.9999)
    x = scale * (0.5 + lon / 360)
    y = scale * (0.5 - np.log((1 + siny) / (1 - siny)) / (4 * np.pi))
    return x, y

def image_size(lat1: float, lon1: float, lat2: float,
    lon2: float, zoom: int, tile_size: int = 256):
    """ Calculates the size of an image without downloading it. Returns the width and height in pixels as a tuple. """

    scale = 1 << zoom
    tl_proj_x, tl_proj_y = project_with_scale(lat1, lon1, scale)
    br_proj_x, br_proj_y = project_with_scale(lat2, lon2, scale)

    tl_pixel_x = int(tl_proj_x * tile_size)
    tl_pixel_y = int(tl_proj_y * tile_size)
    br_pixel_x = int(br_proj_x * tile_size)
    br_pixel_y = int(br_proj_y * tile_size)

    return abs(tl_pixel_x - br_pixel_x), br_pixel_y - tl_pixel_y

In [267]:
Z = 19
R = (256 * 2**Z) / 360  

In [268]:
def calculate_image_coords(lat, lon):
    delta_lon, delta_lat = 4000 / R, 3000 / R

    top_left =  lat + (delta_lat / 2), lon - (delta_lon / 2)
    bottom_right = lat - (delta_lat / 2), lon + (delta_lon / 2)

    return top_left, bottom_right

In [ ]:
def center(lat1, lon1, lat2, lon2):
    return (lat1 + lat2) / 2, (lon1 + lon2) / 2

def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 

    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat / 2) * math.sin(dlat / 2) +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
         math.sin(dlon / 2) * math.sin(dlon / 2))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = radius * c

    return d

In [270]:
def adjust_coords(lat1, lon1, lat2, lon2, zoom, desired_width=4000, desired_height=3000):
    init_center = center(lat1, lon1, lat2, lon2)
    # print(f"Started with: ({lat1}, {lon1}); ({lat2}, {lon2}).")
    tolerance = 0.1   
    adjustment_step = 0.001  
    cnt = 0
    max_iterations = 10**5

    while cnt < max_iterations:
        width, height = image_size(lat1, lon1, lat2, lon2, zoom)

        if abs(width - desired_width) < tolerance and abs(height - desired_height) < tolerance:
            print("Target dimensions achieved!")
            break

        if height < desired_height:
            lat1 = min(lat1 + adjustment_step, 90)  
        else:
            lat1 = max(lat1 - adjustment_step, -90)

        if width < desired_width:
            lon2 = min(lon2 + adjustment_step, 180)  
        else:
            lon2 = max(lon2 - adjustment_step, -180)

        cnt += 1
        adjustment_step = max(0.00001, adjustment_step * 0.99)

        if width > 10 * desired_width or height > 10 * desired_height:
            print("Dimension explosion detected. Aborting adjustments.")
            break

    if cnt >= max_iterations:
        print("Max iterations reached. Adjustments may be incomplete.")
    adjusted_center = center(lat1, lon1, lat2, lon2)
    delta = distance(init_center, adjusted_center)
    print(f"Ended with: ({lat1}, {lon1}); ({lat2}, {lon2}).")
    print(f"Center distorted by {delta:.2f} km.")
    return lat1, lon1, lat2, lon2

In [271]:
init_center_lat, init_center_lon = -33.840902, -61.560890
init_center = init_center_lat, init_center_lon
(lat1, lon1), (lat2, lon2) = calculate_image_coords(init_center_lat, init_center_lon)

In [272]:
lat1, lon1, lat2, lon2 = adjust_coords(lat1, lon1, lat2, lon2, zoom=19)
lat1, lon1, lat2, lon2 = adjust_coords(lat1, lon1, lat2, lon2, zoom=19)

Max iterations reached. Adjustments may be incomplete.
Ended with: (-33.83823214759757, -61.566254418029786); (-33.84492531352234, -61.55552752761792).
Center distorted by 0.08 km.
Target dimensions achieved!
Ended with: (-33.83824214349833, -61.566254418029786); (-33.84492531352234, -61.5555249028938).
Center distorted by 0.00 km.


In [273]:
desired_width, desired_height = 4000, 3000

width, height = image_size(lat1, lon1, lat2, lon2, Z)
print(f"Calculated dimensions: {width}x{height}")

Calculated dimensions: 4000x3000
